DATA643: Project 1
======
Authors: Prashanth Padebettu, Parshu Rath, Adejare Windokun
----
<b>Summer 2016. CUNY School of Professional Studies. <br>
Instructor: Andrew Catlin <br>
<b>

<b> Introduction:
<br><br></b>We used the MovieRatings dataset that was generated by the class for this project. This system recommends movies for a given user. <br>

We make an attempt to use (user-item) collaborative filtering technique for this project.

<br>Typically, the workflow of a collaborative filtering system is: (Source: Wikipedia)<br>

A user expresses his or her preferences by rating items (<i>e.g.</i> books, movies or CDs) of the system. These ratings can be viewed as an approximate representation of the user's interest in the corresponding domain.
The system matches this user’s ratings against other users’ and finds the people with most "similar" tastes.
With similar users, the system recommends items that the similar users have rated highly but not yet being rated by this user (presumably the absence of rating is often considered as the unfamiliarity of an item).<br>
<br>
Python codes for collaborative filtering are shown below.

<b> Load data into pandas dataframe:

In [2]:
#Load data into a pandas dataframe
import csv
#import os
import pandas as pd
import numpy as np
from scipy import spatial
#os.chdir('C:\\Users\ppadebettu\Documents\GitHub\IS-643-Recommender Systems\Week_1')
url = 'https://raw.githubusercontent.com/ppadebettu/CUNY/Master/IS_643_Recommender_Systems/Project_1/MovieRatings.csv'
df = pd.read_csv(url, sep = "," , header = 0, na_values='NaN')

In [3]:
#Display data
df
#df = df.fillna(0)

,Critic,CaptainAmerica,Deadpool,Frozen,JungleBook,PitchPerfect2,StarWarsForce
0,Burton,NaN,NaN,NaN,4.0,NaN,4.0
1,Charley,4.0,5.0,4.0,3.0,2.0,3.0
2,Dan,NaN,5.0,NaN,NaN,NaN,5.0
3,Dieudonne,5.0,4.0,NaN,NaN,NaN,5.0
4,Matt,4.0,NaN,2.0,NaN,2.0,5.0
5,Mauricio,4.0,NaN,3.0,3.0,4.0,NaN
6,Max,4.0,4.0,4.0,2.0,2.0,4.0
7,Nathan,NaN,NaN,NaN,NaN,NaN,4.0
8,Param,4.0,4.0,1.0,NaN,NaN,5.0
9,Parshu,4.0,3.0,5.0,5.0,2.0,3.0


<b> Convert dataframe data to dictionary data:

In [5]:
#Convert the dataframe into dictionary format
#file.set_index('CaptainAmerica').to_dict()
movie_dict = df.set_index('Critic').T.to_dict('dict')

In [6]:
#Display data in the dictionary format
movie_dict['Parshu']

{'CaptainAmerica': 4.0,
 'Deadpool': 3.0,
 'Frozen': 5.0,
 'JungleBook': 5.0,
 'PitchPerfect2': 2.0,
 'StarWarsForce': 3.0}

<b> Euclidean Distance based similarity:

In [13]:
from math import sqrt

#Function that returns a distance-based similarity score for person1 and person2

def sim_distance(dict, p1, p2):
#Get the list of mutually rated items, ignore items that were not rated by either person
 si = {}
 for item in dict[p1]:
       if (item in dict[p2]) and not(np.isnan(dict[p2][item])) and not(np.isnan(dict[p1][item])) :
          si[item] = 1
#If they have no rating in common, return 0
 if len(si) == 0: 
  return 0

#Add up the squares of all differences for common ratings
 sum_of_squares = sum([pow(dict[p1][item]-dict[p2][item],2) for item in si.keys()])
# you have to take the square root of the sum of squares
# see http://stats.stackexchange.com/questions/53068/euclidean-distance-score-and-similarity
 return round(1 / (1 + sqrt(sum_of_squares)), 4)

In [15]:
# Test the distance-based similarity function for two users

sim_distance(movie_dict, 'Prashanth','Parshu')

0.2899

<b> Cosine Distance based similarity:

In [16]:
#Function that returns a cosine-based similarity score for person1 and person2
def sim_cosine(dict, p1,p2):
#Get the list of mutually rated items, ignore items that were not rated by either person
 si = {}
 p1set = []
 p2set = []
 for item in dict[p1]:
       if (item in dict[p2]) and not(np.isnan(dict[p2][item])) and not(np.isnan(dict[p1][item])) :
          si[item] = 1
#If they have no rating in common, return 0
 if len(si) == 0: 
  return 0
 p1set = [dict[p1][item] for item in si.keys()]
 p2set = [dict[p2][item] for item in si.keys()]
 return round((1 - spatial.distance.cosine(p1set, p2set)), 4)

In [18]:
# Test the cosine-based similarity function for two users
round(sim_cosine(movie_dict, 'Prashanth','Parshu'), 4)

0.9827

<b> Similarity based on Pearson correlation

In [25]:
#Function that returns the Pearson correlation coefficient for person1 and person2 
def sim_pearson(dict,p1,p2):
 #Get the list of mutually rated items, ignore items that were not rated by either person
 si = {}
 for item in dict[p1]:
  if item in dict[p2] and not(np.isnan(dict[p2][item])) and not(np.isnan(dict[p1][item])): 
   si[item] = 1

 #if they are no rating in common, return 0
 if len(si) == 0:
  return 0

 #sum calculations
 n = len(si)
            
 #sum of all preferences
 sum1 = sum([dict[p1][it] for it in si])
 sum2 = sum([dict[p2][it] for it in si])
 #Sum of the squares
 sum1Sq = sum([pow(dict[p1][it],2) for it in si])
 sum2Sq = sum([pow(dict[p2][it],2) for it in si])
 #Sum of the products
 pSum = sum([dict[p1][it] * dict[p2][it] for it in si])
 #Calculate r (Pearson score)
 num = pSum - (sum1 * sum2)/n
 den = sqrt((sum1Sq - pow(sum1,2)/n) * (sum2Sq - pow(sum2,2)/n))
 if den == 0:
  return 0

 r = num/den

 return r


In [26]:
# Test the Pearson-based similarity function
round(sim_pearson(movie_dict,'Prashanth','Parshu'), 4)

0.559

<b> Return Top 5 matches

In [38]:
#Function which returns the top 5 best matches for the person from the movie dictionary
#Number of the results and similiraty function are optional parameters
def topMatches(dict,person,n=5,similarity=sim_distance):
     scores = [(similarity(dict,person,other),other) for other in dict if other != person]
     scores.sort()
     scores.reverse()
     return scores[0:n]

In [40]:
# Test top best matches for a person from the movie dictionary
topMatches(movie_dict, 'Parshu', similarity=sim_pearson)

[(0.86602540378443904, 'Shipra'),
 (0.55901699437494845, 'Prashanth'),
 (0.50000000000000266, 'Dieudonne'),
 (0.31491832864888686, 'Charley'),
 (0.10660035817780605, 'Max')]

<b> Recommendations by weighted average:

In [44]:
#Function that gets recommendations for a person by using a weighted average of every other user's rankings

def getRecommendations(dict,person,similarity=sim_distance):
 totals = {}
 simSums = {}

 for other in dict:
  #don't compare me to myself
  if other == person:
   continue
  sim = similarity(dict,person,other)
  #ignore scores of zero or lower
  if sim <= 0: 
   continue
  for item in dict[other]:
   #only score movies I haven't rated yet
   if (item not in dict[person]) or (dict[person][item] == 0) or (np.isnan(dict[person][item])) :
    #Similarity * score
    totals.setdefault(item,0)
    totals[item] += not(np.isnan(dict[other][item] * sim))
    #Sum of similarities
    simSums.setdefault(item,0)
    simSums[item] += sim

    #Create the normalized list
 rankings = [(total/simSums[item],item) for item,total in totals.items()]

 #Return the sorted list
 rankings.sort()
 rankings.reverse()
 return rankings


In [45]:
#Test getRecommendations function
getRecommendations(movie_dict,'Mauricio',similarity=sim_distance)

[(2.167359564951825, 'StarWarsForce'), (1.5762615017831456, 'Deadpool')]

<b> Recommendations using 'Graphlab Create' Package

In [46]:
#Now we validate the results with python graphlab package
#now reshape from wide to long format
#df_long = pd.wide_to_long(df, ['Rating'], i = 'Critic', j = 'Movie')
df_long = pd.melt(df, id_vars=['Critic'])
df_long.columns = ['Critic', 'Movie', 'Rating']
#Remove NaN's as graph lab doesn't seem to like NaN's or zeroes
df_long = df_long.dropna()
df_long["Rating"] = df_long["Rating"].astype(int)

In [47]:
#Display dataframe in long format
df_long.head()

,Critic,Movie,Rating
1,Charley,CaptainAmerica,4
3,Dieudonne,CaptainAmerica,5
4,Matt,CaptainAmerica,4
5,Mauricio,CaptainAmerica,4
6,Max,CaptainAmerica,4


In [50]:
#Let's use graphlab package
import graphlab
from graphlab import SFrame
#Convert pandas dataframe to graphlab SFrame object
sf = graphlab.SFrame(df_long)

In [51]:
#Display SFrame data
sf.head()

Critic,Movie,Rating
Charley,CaptainAmerica,4
Dieudonne,CaptainAmerica,5
Matt,CaptainAmerica,4
Mauricio,CaptainAmerica,4
Max,CaptainAmerica,4
Param,CaptainAmerica,4
Parshu,CaptainAmerica,4
Prashanth,CaptainAmerica,5
Sreejaya,CaptainAmerica,5
Steve,CaptainAmerica,4


In [94]:
#Create recommender model with default similarity_type = jaccard
model = graphlab.recommender.create(sf, user_id='Critic', item_id='Movie')

Recsys training: model = item_similarity

Warning: Column 'Rating' ignored.

To use this column as the target, set target = "Rating" and use a method that allows the use of a target.

Preparing data set.

Data has 61 observations with 16 users and 6 items.

...

| 1.001ms                             | 100              | 6               |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.001001s

In [95]:
#Display model
model

Class                           : ItemSimilarityRecommender

Schema
------
User ID                         : Critic
Item ID                         : Movie
Target                          : None
Additional observation features : 0
Number of user side features    : 0
Number of item side features    : 0

Statistics
----------
Number of observations          : 61
Number of users                 : 16
Number of items                 : 6

Training summary
----------------
Training time                   : 0.001

Model Parameters
----------------
Model class                     : ItemSimilarityRecommender
threshold                       : 0.001
similarity_type                 : jaccard
training_method                 : auto

Other Settings
--------------
degree_approximation_threshold  : 4096
sparse_density_estimation_sample_size: 4096
max_data_passes                 : 4096
target_memory_usage             : 8589934592
seed_item_set_size              : 50
nearest_neighbors_interaction_proporti

In [96]:
##Create recommender model with similarity_type = cosine
model_cosine = graphlab.item_similarity_recommender.create(sf, user_id="Critic", item_id="Movie", target="Rating",
                                                           similarity_type="cosine")

Recsys training: model = item_similarity

Preparing data set.

Data has 61 observations with 16 users and 6 items.

Data prepared in: 0.004004s

Training model from provided data.

...

| 999us                               | 100              | 6               |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.000999s

In [97]:
#Display model
model_cosine

Class                           : ItemSimilarityRecommender

Schema
------
User ID                         : Critic
Item ID                         : Movie
Target                          : Rating
Additional observation features : 0
Number of user side features    : 0
Number of item side features    : 0

Statistics
----------
Number of observations          : 61
Number of users                 : 16
Number of items                 : 6

Training summary
----------------
Training time                   : 0.001

Model Parameters
----------------
Model class                     : ItemSimilarityRecommender
threshold                       : 0.001
similarity_type                 : cosine
training_method                 : auto

Other Settings
--------------
degree_approximation_threshold  : 4096
sparse_density_estimation_sample_size: 4096
max_data_passes                 : 4096
target_memory_usage             : 8589934592
seed_item_set_size              : 50
nearest_neighbors_interaction_proport

In [101]:
#Make some recommedations using survey data from the class based on default similarity type
recommend_result = model.recommend(users=None, k=10)
recommend_result.head()

Critic,Movie,score,rank
Dieudonne,Frozen,0.576923092206,1
Dieudonne,PitchPerfect2,0.474747459094,2
Dieudonne,JungleBook,0.466346144676,3
Matt,JungleBook,0.558238640428,1
Matt,Deadpool,0.557775571942,2
Mauricio,Deadpool,0.530303031206,1
Mauricio,StarWarsForce,0.467708334327,2
Param,JungleBook,0.537259608507,1
Param,PitchPerfect2,0.515151500702,2
Prashanth,PitchPerfect2,0.521212112904,1


In [59]:
#Make some recommedations using survey data from the class based on cosine similarity type
recommend_result = model_cosine.recommend(users=None, k=10)
recommend_result.head()

Critic,Movie,score,rank
Dieudonne,Frozen,3.10371631384,1
Dieudonne,PitchPerfect2,2.73654061556,2
Dieudonne,JungleBook,2.64010673761,3
Matt,Deadpool,2.34341108799,1
Matt,JungleBook,2.01518577337,2
Mauricio,Deadpool,2.31735724211,1
Mauricio,StarWarsForce,2.02999526262,2
Param,JungleBook,2.05938327312,1
Param,PitchPerfect2,2.04626643658,2
Prashanth,PitchPerfect2,2.98132222891,1


### Use Python's built in functions and compare results

In [78]:
# Will redo the import so as not to mess with the code above

url = 'https://raw.githubusercontent.com/ppadebettu/CUNY/Master/IS_643_Recommender_Systems/Project_1/MovieRatings.csv'
dfp = pd.read_csv(url, sep = "," , header = 0, na_values='NaN')
dfp.head()

,Critic,CaptainAmerica,Deadpool,Frozen,JungleBook,PitchPerfect2,StarWarsForce
0,Burton,NaN,NaN,NaN,4.0,NaN,4.0
1,Charley,4.0,5.0,4.0,3.0,2.0,3.0
2,Dan,NaN,5.0,NaN,NaN,NaN,5.0
3,Dieudonne,5.0,4.0,NaN,NaN,NaN,5.0
4,Matt,4.0,NaN,2.0,NaN,2.0,5.0


In [79]:
# Will use the users as the index
dfp.set_index('Critic', inplace = True)
del dfp.index.name
dfp


,CaptainAmerica,Deadpool,Frozen,JungleBook,PitchPerfect2,StarWarsForce
Burton,NaN,NaN,NaN,4.0,NaN,4.0
Charley,4.0,5.0,4.0,3.0,2.0,3.0
Dan,NaN,5.0,NaN,NaN,NaN,5.0
Dieudonne,5.0,4.0,NaN,NaN,NaN,5.0
Matt,4.0,NaN,2.0,NaN,2.0,5.0
Mauricio,4.0,NaN,3.0,3.0,4.0,NaN
Max,4.0,4.0,4.0,2.0,2.0,4.0
Nathan,NaN,NaN,NaN,NaN,NaN,4.0
Param,4.0,4.0,1.0,NaN,NaN,5.0
Parshu,4.0,3.0,5.0,5.0,2.0,3.0


In [80]:
# Have to fill the NaN with 0
# dfp.fillna(0.0, axis =1, inplace = True)
#dfp

In [81]:
# Required Packages from scipy
from scipy.spatial.distance import cosine
from scipy.spatial.distance import correlation
from scipy.spatial.distance import pdist

In [83]:
# Code to take care of the missing values in the dataset

def missing_values(d, p1, p2):
    data = d.loc[d.index.isin([p1, p2])]
    newdata = data.dropna(axis = 1)
    return newdata

<b> Cosine Distance

In [85]:
# Cosine Distance
def sim_cosine_p(d, p1, p2):
 d = missing_values(d, p1, p2)   
 return 1- cosine(d.ix[p1], d.ix[p2])


In [86]:
def sim_cosine_p_pdist(d, p1, p2):
    
 d = missing_values(d, p1, p2) 
 return 1- pdist(d, 'cosine')


In [87]:
# Compare Results
print sim_cosine(movie_dict,'Prashanth','Parshu')
print sim_cosine_p(dfp,'Prashanth','Parshu')
print sim_cosine_p_pdist(dfp,'Prashanth','Parshu')


0.9827
0.982659589399
[ 0.98265959]


<b>Euclidean Distance

In [88]:
def sim_euclidean(d, p1, p2):
    
 d = missing_values(d, p1, p2) 
 s =  pdist(d, 'euclidean')
 return 1/(1+s)   


In [90]:
print sim_distance(movie_dict,'Prashanth','Parshu')
print sim_euclidean(dfp,'Prashanth','Parshu')

0.2899
[ 0.28989795]


<b> Pearson Correlation Simmilarity

In [91]:
def sim_correlation(d, p1, p2):
 d = missing_values(d, p1, p2) 
 return 1- pdist(d, 'correlation')

In [92]:
print sim_pearson(movie_dict,'Prashanth','Parshu')
print sim_correlation(dfp,'Prashanth','Parshu')

0.559016994375
[ 0.55901699]
